In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, BatchNormalization
import seaborn as sns
import smogn
from sklearn.preprocessing import LabelEncoder


# 讀取 Excel 表
df1 = pd.read_excel('/Users/ccit0915/Desktop/迴歸分析/回歸分析期末報告/台中市預售屋買賣資料11204.xlsx')
df2 = pd.read_excel('/Users/ccit0915/Desktop/迴歸分析/回歸分析期末報告/台中市預售屋買賣資料11301.xlsx')
df3 = pd.read_excel('/Users/ccit0915/Desktop/迴歸分析/回歸分析期末報告/台中市預售屋買賣資料11302.xlsx')
df4 = pd.read_excel('/Users/ccit0915/Desktop/迴歸分析/回歸分析期末報告/台中市預售屋買賣資料11303.xlsx')


# 合併 DataFrame
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# 顯示合併後的 DataFrame
print(df)

                   /預售屋/土地位置建物門牌  /預售屋/土地移轉總面積平方公尺  /預售屋/土地移轉總面積平方公尺/#agg  \
0                     臺中市北屯區祥順一街            105.70                 105.70   
1      臺中市新社區水底寮段上水底寮小段213地號等18筆            113.73                 113.73   
2              臺中市豐原區鎌村路107巷102弄             20.89                  20.89   
3              臺中市豐原區鎌村路107巷102弄             20.74                  20.74   
4                      臺中市梧棲區中華路             24.02                  24.02   
...                          ...               ...                    ...   
29106          臺中市太平區東平段191-12地號             79.73                  79.73   
29107          臺中市太平區東平段191-13地號             82.44                  82.44   
29108             臺中市西屯區惠民段114地號              9.58                   9.58   
29109             臺中市西屯區惠民段114地號              9.58                   9.58   
29110             臺中市西屯區惠民段114地號              6.73                   6.73   

      /預售屋/主要用途 /預售屋/主要建材  /預售屋/交易年月日  /預售屋/交易年月日/#agg /預售屋/交易筆棟數  \
0     

In [8]:
df.drop(columns=['/預售屋/土地移轉總面積平方公尺/#agg'], inplace=True)
df.drop(columns=['/預售屋/交易年月日/#agg'], inplace=True)
df.drop(columns=['/預售屋/單價元平方公尺/#agg'], inplace=True)
df.drop(columns=['/預售屋/總價元/#agg'], inplace=True)
df.drop(columns=['/預售屋/總樓層數/#agg'], inplace=True)
df.drop(columns=['/預售屋/車位移轉總面積平方公尺/#agg'], inplace=True)
df.drop(columns=['/預售屋/車位總價元/#agg'], inplace=True)
df.drop(columns=['/預售屋/建物現況格局-房/#agg'], inplace=True)
df.drop(columns=['/預售屋/建物現況格局-廳/#agg'], inplace=True)
df.drop(columns=['/預售屋/建物現況格局-衛/#agg'], inplace=True)
df.drop(columns=['/預售屋/建物移轉總面積平方公尺/#agg'], inplace=True)
df.drop(columns=['/預售屋/土地位置建物門牌'], inplace=True)
df.drop(columns=['/預售屋/有無管理組織'], inplace=True)
df.drop(columns=['/預售屋/棟及號'], inplace=True)
df.drop(columns=['/預售屋/解約情形'], inplace=True)
df.drop(columns=['/預售屋/編號'], inplace=True)
df.drop(columns=['/預售屋/建案名稱'], inplace=True)
df.drop(columns=['/預售屋/建築完成年月'], inplace=True)
df.drop(columns=['/預售屋/備註'], inplace=True)
df.drop(columns=['/預售屋/非都市土地使用分區'], inplace=True)
df.drop(columns=['/預售屋/非都市土地使用編定'], inplace=True)
df.drop(columns=['/預售屋/交易年月日'], inplace=True)

In [10]:
# 檢查缺失值
missing_values = df.isnull().sum()

# 顯示有缺失值的欄位
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

/預售屋/車位類別        2455
/預售屋/都市土地使用分區    1273
/預售屋/單價元平方公尺       10
/預售屋/鄉鎮市區          11
dtype: int64


In [12]:
# 刪除目標變數缺失值的10筆記錄
df = df.dropna(subset=["/預售屋/單價元平方公尺"])

# 將缺失值填入 "0" 表示無車位
df["/預售屋/車位類別"] = df["/預售屋/車位類別"].fillna("0")

# 以眾數填入缺失值
mode_value = df["/預售屋/都市土地使用分區"].mode()[0]
df["/預售屋/都市土地使用分區"] = df["/預售屋/都市土地使用分區"].fillna(mode_value)

# 刪除/預售屋/鄉鎮市區的缺失11筆記錄
df = df.dropna(subset=["/預售屋/鄉鎮市區"])

# 檢查缺失值
missing_values = df.isnull().sum()

# 顯示有缺失值的欄位
missing_columns = missing_values[missing_values > 0]
print(missing_columns)

Series([], dtype: int64)


In [14]:
df.drop(columns=['/預售屋/總價元'], inplace=True)

In [16]:
# 定義中英對應字典
column_name_map = {
    '/預售屋/土地移轉總面積平方公尺': 'Land Transfer Area (sqm)',
    '/預售屋/主要用途': 'Main Use',
    '/預售屋/主要建材': 'Main Materials',
    '/預售屋/交易筆棟數': 'Transaction Volume',
    '/預售屋/交易標的': 'Transaction Target',
    '/預售屋/車位移轉總面積平方公尺': 'Parking Transfer Area (sqm)',
    '/預售屋/車位總價元': 'Parking Price',
    '/預售屋/車位類別': 'Parking Type',
    '/預售屋/建物型態': 'Building Type',
    '/預售屋/建物現況格局-房': 'House Layout - Room',
    '/預售屋/建物現況格局-隔間': 'House Layout - Partition',
    '/預售屋/建物現況格局-衛': 'House Layout - Bathroom',
    '/預售屋/建物現況格局-廳': 'House Layout - Living Room',
    '/預售屋/建物移轉總面積平方公尺': 'Building Transfer Area (sqm)',
    '/預售屋/移轉層次': 'Transfer Level',
    '/預售屋/都市土地使用分區': 'Urban Land Use Zone',
    '/預售屋/單價元平方公尺': 'Unit Price (per sqm)',
    '/預售屋/鄉鎮市區': 'Township/District',
    '/預售屋/總樓層數': 'Total Floors'
}
df.rename(columns=column_name_map, inplace=True)

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# 定義特徵和目標變數
X = df.drop(columns=['Unit Price (per sqm)'])  # 將目標變數從特徵集中移除
y = df['Unit Price (per sqm)']                # 定義目標變數

# 切割數據集，80% 用於訓練，20% 用於測試
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 假設原始資料在 DataFrame 中，地區欄位為 "Township/District"
freq_map_township = (
    X_train["Township/District"]
    .value_counts(normalize=True)  # 計算頻率
    .to_dict()  # 轉換為字典
)

# 初始化 LabelEncoder
label_encoder = LabelEncoder()

# 1. 標籤編碼（修正未知類別處理）
categorical_features = [
    "Main Use",
    "Main Materials",
    "Transaction Target",
    "House Layout - Partition",
    "Urban Land Use Zone"
]
# 1. 標籤編碼（將未知類別填補為 1）
for feature in categorical_features:
    # 對訓練集進行標籤編碼
    X_train[f"{feature}_encoded"] = label_encoder.fit_transform(X_train[feature].copy())
    # 測試集應用相同的映射，對未見過的類別填補為 -1
    label_mapping = {label: idx for idx, label in enumerate(label_encoder.classes_)}
    X_test[f"{feature}_encoded"] = X_test[feature].map(label_mapping).fillna(-1)

# 2. 頻率編碼
frequency_features = [
    "Transaction Volume",
    "Building Type",
    "Township/District", 
    "Transfer Level"
]

for feature in frequency_features:
    # 計算頻率映射表
    freq_map = X_train[feature].value_counts(normalize=True)
    # 對訓練集應用頻率映射
    X_train[f"{feature}_freq"] = X_train[feature].map(freq_map).fillna(1)  # 填補為 1
    # 對測試集應用頻率映射，對未知值填補為 -1
    X_test[f"{feature}_freq"] = X_test[feature].map(freq_map).fillna(-1)  # 填補為 1


# 3. 目標編碼
target_encoding_features = [("Parking Type", X_train["Parking Price"])]

for feature, target in target_encoding_features:
    # 計算基於 Parking Price 的目標編碼均值
    target_map = X_train.groupby(feature)[target.name].mean()
    # 對訓練集進行目標編碼
    X_train[f"{feature}_encoded"] = X_train[feature].map(target_map)
    # 測試集目標編碼，對未知值填補為整體 Parking Price 均值
    X_test[f"{feature}_encoded"] = X_test[feature].map(target_map).fillna(target.mean())

# 移除原始特徵以避免資料洩漏
X_train = X_train.drop(columns=categorical_features + frequency_features)
X_test = X_test.drop(columns=categorical_features + frequency_features)

In [20]:
X_train.drop(columns=['Parking Type'], inplace=True)
X_test.drop(columns=['Parking Type'], inplace=True)

In [22]:
freq_map_township

{'北屯區': 0.17600550017188038,
 '西屯區': 0.14644207631488484,
 '太平區': 0.09543657614300446,
 '南屯區': 0.0812564455139223,
 '沙鹿區': 0.06974046063939499,
 '烏日區': 0.06342385699553112,
 '梧棲區': 0.059470608456514265,
 '南區': 0.052939154348573395,
 '清水區': 0.03983327603987625,
 '大里區': 0.03235647988999656,
 '龍井區': 0.03016500515641114,
 '豐原區': 0.022172567892746648,
 '北區': 0.021055345479546236,
 '潭子區': 0.018863870745960812,
 '后里區': 0.013234788587143349,
 '大甲區': 0.012332416638019938,
 '東區': 0.010570642832588519,
 '外埔區': 0.010097971811619113,
 '西區': 0.010055001718803712,
 '神岡區': 0.009324510140941905,
 '大雅區': 0.007047095221725679,
 '大肚區': 0.006101753179786868,
 '中區': 0.004640770024063252,
 '霧峰區': 0.0024063251976624267,
 '東勢區': 0.0021914747335854247,
 '大安區': 0.0014609831557236164,
 '新社區': 0.001074252320385012,
 '石岡區': 0.00030079064970780334}

In [24]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# 定義前五個重要特徵
important_features = [
    'Township/District_freq',  # 重要性：0.630906
    'Parking Price',           # 重要性：0.165049
    'Total Floors',            # 重要性：0.057787
    'Land Transfer Area (sqm)', # 重要性：0.035245
    'Building Transfer Area (sqm)' # 重要性：0.022468
]

# 使用這些特徵作為 X
X_train_subset = X_train[important_features]
X_test_subset = X_test[important_features]

# 創建並訓練隨機森林回歸模型
model = RandomForestRegressor(random_state=42)
model.fit(X_train_subset, y_train)

# 預測
y_train_pred = model.predict(X_train_subset)
y_test_pred = model.predict(X_test_subset)

# 計算 R^2 分數
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# 計算調整後的 R²
n_train = X_train_subset.shape[0]  # 訓練集樣本數
n_test = X_test_subset.shape[0]    # 測試集樣本數
p = X_train_subset.shape[1]        # 特徵數

train_ra2 = 1 - (1 - train_r2) * (n_train - 1) / (n_train - p - 1)
test_ra2 = 1 - (1 - test_r2) * (n_test - 1) / (n_test - p - 1)

# 顯示結果
print(f"訓練集 R²：{train_r2:.4f}")
print(f"訓練集 Ra²：{train_ra2:.4f}")
print(f"測試集 R²：{test_r2:.4f}")
print(f"測試集 Ra²：{test_ra2:.4f}")

訓練集 R²：0.9901
訓練集 Ra²：0.9901
測試集 R²：0.9614
測試集 Ra²：0.9614


In [26]:
import pandas as pd
from ipywidgets import Layout
from ipywidgets import interact, Dropdown, FloatText, IntText, fixed

# freq_map_township 定義
freq_map_township = {
    '北屯區': 0.17600550017188038,
    '西屯區': 0.14644207631488484,
    '太平區': 0.09543657614300446,
    '南屯區': 0.0812564455139223,
    '沙鹿區': 0.06974046063939499,
    '烏日區': 0.06342385699553112,
    '梧棲區': 0.059470608456514265,
    '南區': 0.052939154348573395,
    '清水區': 0.03983327603987625,
    '大里區': 0.03235647988999656,
    '龍井區': 0.03016500515641114,
    '豐原區': 0.022172567892746648,
    '北區': 0.021055345479546236,
    '潭子區': 0.018863870745960812,
    '后里區': 0.013234788587143349,
    '大甲區': 0.012332416638019938,
    '東區': 0.010570642832588519,
    '外埔區': 0.010097971811619113,
    '西區': 0.010055001718803712,
    '神岡區': 0.009324510140941905,
    '大雅區': 0.007047095221725679,
    '大肚區': 0.006101753179786868,
    '中區': 0.004640770024063252,
    '霧峰區': 0.0024063251976624267,
    '東勢區': 0.0021914747335854247,
    '大安區': 0.0014609831557236164,
    '新社區': 0.001074252320385012,
    '石岡區': 0.00030079064970780334
}

# 模型預測函數
def predict_with_model_ui(township_name, parking_price, total_floors, land_area, building_area, model):
    """
    使用模型進行預測，並將每平方公尺價格轉為每坪價格。
    """
    # 處理 Township Name
    township_freq = freq_map_township.get(township_name, None)
    if township_freq is None:
        raise ValueError(f"地區名稱 '{township_name}' 不在映射表中！請檢查輸入。")

    # 使用輸入的車位價格
    # 當車位價格為0時表示沒有車位需求

    # 構建輸入數據
    input_data = pd.DataFrame([{
        'Township/District_freq': township_freq,
        'Parking Price': parking_price,
        'Total Floors': total_floors,
        'Land Transfer Area (sqm)': land_area,
        'Building Transfer Area (sqm)': building_area
    }])

    # 使用模型進行預測
    prediction_per_sqm = model.predict(input_data)[0]


    return f"試算結果：每平方公尺價格為 {prediction_per_sqm:.2f} 元"

# 下拉選單和互動式輸入
def interactive_prediction(model):
    # 創建UI元件
    parking_price = FloatText(
        value=0,
        description="車位預算"
    )

    # 使用interact創建互動式UI
    interactive_ui = interact(
        predict_with_model_ui,
        township_name=Dropdown(
            options=list(freq_map_township.keys()),
            description="地區名稱"
        ),
        parking_price=parking_price,
        total_floors=IntText(
            value=15,
            description="總樓層數"
        ),
        land_area=FloatText(
            value=50.0,
            description="土地移轉面積(平方公尺)",
        ),
        building_area=FloatText(
            value=80.0,
            description="建物移轉面積(平方公尺)",
        ),
        model=fixed(model)
    )

# 試算
interactive_prediction(model=model)

interactive(children=(Dropdown(description='地區名稱', options=('北屯區', '西屯區', '太平區', '南屯區', '沙鹿區', '烏日區', '梧棲區', '…